In [ ]:
import numpy as np
import pandas as pd
import math
from datetime import datetime
import CMAPSAuxFunctions

from data_handler_VALVE import ValveDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

<h1> Create Data Handler </h1>

In [ ]:
features = ['timestamp', 'externalControllerOutput', 'undisturbedMediumFlow', 'pressureValveInlet', 'pressureValveOutlet', 'mediumTemperature', 'rodDisplacement', 'disturbedMediumFlow', 'selectedFault', 'faultType', 'faultIntensity']
selected_indices = np.array([2,4,5,6,7,8])
selected_features = list(features[i] for i in selected_indices-1)

window_size = 30
window_stride = 1

# min = 2018-02-14 18:59:20
# max = 2018-08-19 18:28:20
time_start = "2018-02-14 18:59:20"
time_end = "2018-04-19 18:28:20"

# Either classification or regression
problem = 'classification'
#problem = 'regression'

vHandler = ValveDataHandler(time_start, time_end, selected_features = selected_features,
                            sequence_length = window_size, sequence_stride = window_stride,
                            problem = problem)

<h1> Keras Model </h1>

In [ ]:
#np.set_printoptions(threshold=np.nan)

K.clear_session()
lambda_regularization = 0.20

def create_ANN_model(input_shape, problem):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim = input_shape, activation = 'relu', kernel_initializer = 'glorot_normal', 
                    kernel_regularizer = regularizers.l2(lambda_regularization), name = 'fc1'))
    
    if (problem == 'classification'):
        model.add(Dense(1, activation = 'softmax', name = 'out'))
    elif (problem == 'regression'):
        model.add(Dense(1, activation = 'linear', name = 'out'))
        
    #model.add(Dense(1, activation = 'softmax', name = 'out'))
    
    return model

<h1>Tunable Model </h1>

In [ ]:
scaler = MinMaxScaler(feature_range = (-1, 1))
#scaler = StandardScaler()

In [ ]:
def get_compiled_model(shape, problem):
    
    K.clear_session()
    
    if (problem == 'classification'):
        # Parameters for the model
        optimizer = SGD(lr = 0.01, momentum = 0.0, decay = 0.0, nesterov = False)
        loss_function = 'categorical_crossentropy'
        metrics = ['accuracy']
    elif (problem == 'regression'):
        # Parameters for the model
        optimizer = Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad = False)
        loss_function = 'mean_squared_error'
        metrics = ['mse']
        
    model = None
    
    # Create and compile the model
    model = create_ANN_model(shape, problem)
    model.compile(optimizer = optimizer, loss = loss_function, metrics = metrics)
    
    return model

In [ ]:
num_features = len(selected_features)
input_shape = num_features * window_size

model = get_compiled_model(input_shape, problem)
tModel = SequenceTunableModelRegression('ANN_Model', model, lib_type = 'keras', data_handler = vHandler)

tModel.data_handler.data_scaler = None
tModel.data_scaler = scaler

<h1> Loading Data from MySQL Database </h1>

In [ ]:
#vHandler.connect_to_db("remoteAdmin","remoteAdmin","169.236.181.40:3306","damadics")

In [ ]:
# Extract data from database
#vHandler.extract_data_from_db()
# vHandler.extract_data_from_db()

In [ ]:
# Extract data from .csv file. If loading from .csv, don't forget to uncomment .extract_data_from_db() in data_handler_VALVE'''
vHandler._df = pd.read_csv('valve_dataset.csv', sep = ',')
for i in range(1,9):
    vHandler._df[vHandler._df.columns[-i]] = vHandler._df[vHandler._df.columns[-i-1]]
vHandler._df['undisturbedMediumFlow'] = pd.np.nan

if (problem == 'classification'):
    # For anomaly detection, the standard is to indicate inliners as '1' and outliers as '-1'
    vHandler._df['status'] = vHandler._df['selectedFault'].apply(lambda valve: 1 if valve == 20 else -1)
elif (problem == 'regression'):
    vHandler._y = vHandler._df["selectedFault"].values

# adds a new feature called "Status" on whether the valve is broken '1' or not broken '0'
#vHandler._df['status'] = vHandler._df['selectedFault'].apply(lambda valve: 0 if valve == 20 else 1)
# this adds a column to vHandler._X
#vHandler._X = vHandler._df[selected_features]
#vHandler._X.insert(loc = 0, column = 'id', value = np.arange(len(vHandler._df)))
# vHandler._X = vHandler._X.values

# vHandlezr._y = vHandler._df['status'].values
# print(vHandler._df.head())

In [ ]:
# vHandler._df['id'] = np.arange(1, vHandler._df.shape[0] + 1)
# print(vHandler._df)

In [ ]:
#scaler = MinMaxScaler(feature_range = (-1, 1))
scaler = StandardScaler()

vHandler._X = vHandler._df[selected_features].values

old_X = tModel.data_handler._X
tModel.data_handler._X = scaler.fit_transform(tModel.data_handler._X)

print('Data Set')
print('Before: {}'.format(old_X[0][0:5]))
print('After: {}\n'.format(tModel.data_handler._X[0][0:5]))

if (problem == 'classification'):
    vHandler._y = vHandler._df["status"].values
elif (problem == 'regression'):
    vHandler._y = vHandler._df["selectedFault"].values
        
# vHandler._X = vHandler._df[selected_features].values
# vHandler._y = vHandler._df["selectedFault"].values
#vHandler._y = list(map(lambda valve: 0 if valve == 20 else 1, vHandler._y))
#vHandler._y = vHandler._df["status"].values

In [ ]:
# print(vHandler._X)
# print(vHandler._y)

In [ ]:
#vHandler.load_data(cross_validation_ratio = 0.3, test_ratio = 0.2, unroll = True)
tModel.load_data(unroll = True, verbose = 0, cross_validation_ratio = 0.2, test_ratio = 0.2)

In [ ]:
#TODO: add this class onto data_handler_VALVE

# if (problem == 'classification'):
broken_counter = 0
not_broken_counter = 0
dataset = vHandler._y_crossVal
print(dataset.shape)
#print(vHandler._X_test[1])
# print(vHandler._y_test)

for i in dataset:
    #print(i)
    if (i == 1.0):
        not_broken_counter += 1;
    else:
        broken_counter += 1;

print("Broken Counter:", broken_counter)
print("Not Broken Counter:", not_broken_counter)

broken_counter = 0
not_broken_counter = 0
dataset = vHandler._y_test
print(dataset.shape)
#print(vHandler._X_test[1])
# print(vHandler._y_test)

for i in dataset:
    #print(i)
    if (i == 1.0):
        not_broken_counter += 1;
    else:
        broken_counter += 1;

print("Broken Counter:", broken_counter)
print("Not Broken Counter:", not_broken_counter)

In [ ]:
#vHandler.print_data(True)

<h1> One Class SVM </h1>

In [ ]:
from sklearn.svm import OneClassSVM

In [ ]:
start_time = datetime.now()
svm = OneClassSVM(kernel='rbf', degree=5, gamma='scale',
                  coef0=0.0, tol=0.001, nu=0.5,
                  shrinking=True, cache_size=200, verbose=True,
                  max_iter=-1)
svm.fit(tModel.data_handler._X_train)
print('Time Elapsed: {}'.format(datetime.now()- start_time))

In [ ]:
y_pred_crossVal = svm.predict(tModel.data_handler._X_crossVal)
y_pred_test = svm.predict(tModel.data_handler._X_test)

In [ ]:
acc_crossVal = metrics.accuracy_score(tModel.data_handler._X_crossVal, y_pred_crossVal)
precision_crossVal = metrics.precision_score(tModel.data_handler._X_crossVal, y_pred_crossVal)
recall_crossVal = metrics.recall_score(tModel.data_handler._X_crossVal, y_pred_crossVal)
confusion_matrix_crossVal = metrics.confusion_matrix(tModel.data_handler._X_crossVal, y_pred_crossVal)

print("Accuracy for cross-validation:", acc_crossVal)
print("Precision for cross-validation:", precision_crossVal)
print("Recall for cross-validation:", recall_crossVal)
print("Confusion matrix for cross-validation\n", confusion_matrix_crossVal)

In [ ]:
acc_test = metrics.accuracy_score(tModel.data_handler._X_test, y_pred_test)
precision_test = metrics.precision_score(tModel.data_handler._X_test, y_pred_test)
recall_test = metrics.recall_score(tModel.data_handler._X_test, y_pred_test)
confusion_matrix_test = metrics.confusion_matrix(tModel.data_handler._X_test, y_pred_test)

print("Accuracy for test:", acc_test)
print("Precision for test:", precision_test)
print("Recall for test:", recall_test)
print("Confusion matrix for test\n", confusion_matrix_test)

In [ ]:
del svm

<h1> Isolation Forest </h1>

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
# default
# IsolationForest(n_estimators=100, max_samples='auto', contamination='legacy',
#                 max_features=1.0, bootstrap=False, n_jobs=None,
#                 behaviour=’old’, random_state=None, verbose=0)

start_time = datetime.now()
iso_forest = IsolationForest(n_estimators = 100, max_samples = 'auto', contamination = 'legacy',
                             max_features = 1.0, bootstrap = False, n_jobs = None, behaviour = 'new',
                             random_state = None, verbose = 1)
iso_forest.fit(tModel.data_handler._X_train)
print('Time Elapsed: {}'.format(datetime.now()- start_time))

In [ ]:
# iso_forest.decision_function(tModel.data_handler._X_train)

In [ ]:
y_pred_crossVal = iso_forest.predict(tModel.data_handler._X_crossVal)
y_pred_test = iso_forest.predict(tModel.data_handler._X_test)

In [ ]:
acc_crossVal = metrics.accuracy_score(tModel.data_handler._y_crossVal, y_pred_crossVal)
precision_crossVal = metrics.precision_score(tModel.data_handler._y_crossVal, y_pred_crossVal)
recall_crossVal = metrics.recall_score(tModel.data_handler._y_crossVal, y_pred_crossVal)
confusion_matrix_crossVal = metrics.confusion_matrix(tModel.data_handler._y_crossVal, y_pred_crossVal)

print("Accuracy for cross-validation:", acc_crossVal)
print("Precision for cross-validation:", precision_crossVal)
print("Recall for cross-validation:", recall_crossVal)
print("Confusion matrix for cross-validation\n", confusion_matrix_crossVal)

In [ ]:
acc_test = metrics.accuracy_score(tModel.data_handler._y_test, y_pred_test)
precision_test = metrics.precision_score(tModel.data_handler._y_test, y_pred_test)
recall_test = metrics.recall_score(tModel.data_handler._y_test, y_pred_test)
confusion_matrix_test = metrics.confusion_matrix(tModel.data_handler._y_test, y_pred_test)

print("Accuracy for test:", acc_test)
print("Precision for test:", precision_test)
print("Recall for test:", recall_test)
print("Confusion matrix for test\n", confusion_matrix_test)

In [ ]:
del iso_forest

<h1> Robust Covariance/Elliptic Envelope </h1>

In [ ]:
from sklearn.covariance import EllipticEnvelope

In [ ]:
start_time = datetime.now()
EE = EllipticEnvelope(store_precision=True, assume_centered=False, support_fraction=None,
                      contamination=0.1, random_state=None)
EE.fit(tModel.data_handler._X_train)
print('Time Elapsed: {}'.format(datetime.now()- start_time))

In [ ]:
y_pred_crossVal = EE.predict(tModel.data_handler._X_crossVal)
y_pred_test = EE.predict(tModel.data_handler._X_test)

In [ ]:
acc_crossVal = metrics.accuracy_score(tModel.data_handler._y_crossVal, y_pred_crossVal)
precision_crossVal = metrics.precision_score(tModel.data_handler._y_crossVal, y_pred_crossVal)
recall_crossVal = metrics.recall_score(tModel.data_handler._y_crossVal, y_pred_crossVal)
confusion_matrix_crossVal = metrics.confusion_matrix(tModel.data_handler._y_crossVal, y_pred_crossVal)

print("Accuracy for cross-validation:", acc_crossVal)
print("Precision for cross-validation:", precision_crossVal)
print("Recall for cross-validation:", recall_crossVal)
print("Confusion matrix for cross-validation\n", confusion_matrix_crossVal)

In [ ]:
acc_test = metrics.accuracy_score(tModel.data_handler._y_test, y_pred_test)
precision_test = metrics.precision_score(tModel.data_handler._y_test, y_pred_test)
recall_test = metrics.recall_score(tModel.data_handler._y_test, y_pred_test)
confusion_matrix_test = metrics.confusion_matrix(tModel.data_handler._y_test, y_pred_test)

print("Accuracy for test:", acc_test)
print("Precision for test:", precision_test)
print("Recall for test:", recall_test)
print("Confusion matrix for test\n", confusion_matrix_test)